In [1]:
import numpy as np
import tqdm 
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import colors
import matplotlib as mpl 
import sys

sys.path.append('../pipeline_scripts/')
from pipeline_main import pipeline, dist
from pipeline_main_nosink import pipeline_nosink, dist

import osyris
from osyris.plot.direction import get_direction

# Set som plotting standards:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 16}
mpl.rc('font', **font)


In [2]:
data = pipeline(snap = 400, run = 'sink122core02', sink_id = 122)

Initialising patch data
Assigning relative cartesian velocities and coordinates to all cells
Assigning masses to all cells
Calculating adiabatic index γ and pressure (polytropic) for all cells


100%|██████████| 5026/5026 [00:39<00:00, 128.31it/s]


In [13]:
def Dispatch_to_1Dvector(self, variables, vectors = None, verbose = 1):
    if verbose > 0: print('Looping over DISPATCH data to extract data at highest level')
    pp = [p for p in self.sn.patches]
    w= np.array([p.level for p in pp]).argsort()[::-1]
    sorted_patches = [pp[w[i]] for i in range(len(pp))]

    patch_cartcoor = []
    patch_ds = []
    patch_values = {key: [] for key in range(len(variables))}
    if vectors is not None: patch_vectors = {key: [] for key in range(len(vectors))}
    else: patch_vectors = None

    for p in tqdm.tqdm(sorted_patches, disable = not self.loading_bar):
        nbors = [self.sn.patchid[i] for i in p.nbor_ids if i in self.sn.patchid]
        children = [ n for n in nbors if n.level == p.level + 1]
        leafs = [n for n in children if ((n.position - p.position)**2).sum() < ((p.size)**2).sum()/12]
        if len(leafs) == 8: continue
        to_extract = np.ones(p.n, dtype=bool)
        for lp in leafs:
            leaf_extent = np.vstack((lp.position - 0.5 * lp.size, lp.position + 0.5 * lp.size)).T
            covered_bool = ~np.all((p.xyz > leaf_extent[:, 0, None, None, None]) & (p.xyz < leaf_extent[:, 1, None, None, None]), axis=0)
            to_extract *= covered_bool

        #Looping over the data to extract from DISPATCH
        for i, ivs in enumerate(variables):
            if hasattr(p, ivs):
                new_value = getattr(p, ivs)[to_extract].T               
            else:
                new_value = p.var(ivs)[to_extract].T
            patch_values[i].extend(new_value.tolist())

        if vectors is not None:
            for i, ivs in enumerate(vectors):
                new_vector = getattr(p, ivs)[:, to_extract].T
                patch_vectors[i].extend(new_vector.tolist())
     
        #Extracting the position and cellsize data each is fixed across all data
        #The line below should probably be changes to p.xyz to not have the star in the cetner since the code doesn't deal with periodic boundaries
        new_xyz = p.rel_xyz[:,to_extract].T 
        new_xyz[new_xyz < -0.5] += 1
        new_xyz[new_xyz > 0.5] -= 1
        patch_ds.extend(p.ds[0] * np.ones(len(new_value)))
        patch_cartcoor.extend(new_xyz.tolist())
    
    patch_ds = np.asarray(patch_ds)
    patch_cartcoor = np.asarray(patch_cartcoor)
    for key in patch_values:
        patch_values[key] = np.array(patch_values[key])
    if vectors is not None:
        for key in patch_vectors:
            patch_vectors[key] = np.array(patch_vectors[key])
    
    return patch_cartcoor, patch_ds, patch_values, patch_vectors

pipeline.Dispatch_to_1Dvector = Dispatch_to_1Dvector

In [4]:
for p in tqdm.tqdm(data.sn.patches):
    p.B = np.concatenate([p.var(f'b'+axis)[None,...] for axis in ['x','y','z']], axis = 0)

100%|██████████| 5026/5026 [00:08<00:00, 587.90it/s]


In [5]:
variable = ['d']
vector = ['vrel', 'B']

In [14]:
patch_cartcoor, patch_ds, patch_values, patch_vectors = data.Dispatch_to_1Dvector(variables=variable, vectors = vector)

Looping over DISPATCH data to extract data at highest level


100%|██████████| 5026/5026 [00:49<00:00, 102.20it/s]


In [15]:
patch_cartcoor.shape, patch_ds.shape, patch_values[0].shape, patch_vectors[0].shape

((18013696, 3), (18013696,), (18013696,), (18013696, 3))

In [16]:
patch_cartcoor.min(), patch_cartcoor.max()

(-0.4968490632361263, 0.4970474150685993)

In [10]:
patch_cartcoor[patch_cartcoor < -0.5] += 1
patch_cartcoor[patch_cartcoor > 0.5] -= 1

In [11]:
patch_cartcoor.min(), patch_cartcoor.max()

(-0.4968490632361263, 0.4970474150685993)

array([[-2.24114742e-05, -8.34496802e-05, -2.24205942e-05],
       [-2.24114742e-05, -8.34496802e-05, -2.14669199e-05],
       [-2.24114742e-05, -8.34496802e-05, -2.05132456e-05],
       ...,
       [ 2.76718138e-02, -2.85911563e-01,  1.26724151e-02],
       [ 2.76718138e-02, -2.85911563e-01,  2.04849151e-02],
       [ 2.76718138e-02, -2.85911563e-01,  2.82974151e-02]])